In [92]:
import keras
import numpy as np
from numpy import load
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization

In [117]:
#unpack data from source

data = np.load('train_and_test.npz')  #what kind of file is this? zipped np.array file

X = [data[key] for key in data]  #??? why does this work? "data" is not a dictionary data type but a NpzFile type

xtrain0 = X[0]
ytrain0 = X[1]  #ytrain0 contains values between 0-42; so 43 categories
xtest0 = X[2]   #this is test data for final kaggle submission

x_tr, x_tst, y_tr, y_tst = train_test_split(xtrain0, ytrain0, test_size = 0.2,shuffle = True)  #results overfit if data is not shuffled!!!

In [118]:
#build model -  this is a function to build the model where activation type and batch
#normalization can be specified as parameters so models can be changed without retyping

def build_model(activation, batch_normalization = False):
    model = Sequential()
    model.add(Conv2D(8, (3,3), activation = activation, padding = 'same', name = 'conv1', 
                    input_shape = (32,32,3))) 
    if batch_normalization: 
        model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2), name = 'maxpool1')) 
    model.add(Conv2D(16, (3,3), activation = activation, padding = 'same', name = 'conv2' ))
    if batch_normalization: 
        model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2), name = 'maxpool2'))
    model.add(Flatten())
    model.add(Dense(64, activation=activation, name='dense_1'))
    model.add(Dense(43, activation='softmax', name='output'))  #there are 43 labels
    return model

# Optimizing Hyperparameters

The remaining of this notebook will be dedicated to testing for the optimal values of our hyperparameters within our model (specifically our learning rate and our momentum). For our own sanity, we will be setting our number of epochs to 20 for our initial tests to limit processing time. Once a proper learning rate has been selected, we will shift our focus to finding the optimal momentum rate. Finally, we will look for the proper number of epochs to run based off our selected learning rate and momentum.

In [20]:
learning_rates = []
momentums = []

rate = 1
m = 0.99
for i in range(5):
    if rate > 0.125:
        rate = rate/2
    elif rate == 0.125:
        rate = 0.01
    else:
        rate = rate/10
        
    momentums.append(round(m,2))
    m -= 0.02
    learning_rates.append(rate)
    
momentums += [0.5, 0.25, 0.1, 0]

test = [0.1,0.25,0.9,0.95,0.99]
    
print(learning_rates)
print(momentums)

[0.5, 0.25, 0.125, 0.01, 0.001]
[0.99, 0.97, 0.95, 0.93, 0.91, 0.5, 0.25, 0.1, 0]


In [21]:
for i in range(len(learning_rates)):
    print('learning rate:', learning_rates[i])
    print('momentum:', test[i])
    model = build_model(activation = 'sigmoid', batch_normalization = False)  # try 'relu' too
    opt = tf.keras.optimizers.SGD(learning_rate = learning_rates[i], momentum = test[i], nesterov=False)
    model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
    model.fit(x_tr, y_tr, epochs = 20, validation_data= (x_tst, y_tst))
    print('========================================================================================================')

learning rate: 0.5
momentum: 0.1
Train on 26270 samples, validate on 12939 samples
Epoch 1/20
26270/26270 [==============================] - 11s 416us/step - loss: 2.7336 - accuracy: 0.2229 - val_loss: 2.2082 - val_accuracy: 0.3273
Epoch 2/20
26270/26270 [==============================] - 11s 406us/step - loss: 2.0037 - accuracy: 0.3645 - val_loss: 1.7795 - val_accuracy: 0.4207
Epoch 3/20
26270/26270 [==============================] - 11s 413us/step - loss: 1.7876 - accuracy: 0.4243 - val_loss: 1.6838 - val_accuracy: 0.4667
Epoch 4/20
26270/26270 [==============================] - 11s 428us/step - loss: 1.6081 - accuracy: 0.4787 - val_loss: 1.7819 - val_accuracy: 0.4538
Epoch 5/20
26270/26270 [==============================] - 13s 488us/step - loss: 1.6143 - accuracy: 0.4858 - val_loss: 1.4756 - val_accuracy: 0.5313
Epoch 6/20
26270/26270 [==============================] - 11s 434us/step - loss: 1.4448 - accuracy: 0.5416 - val_loss: 1.3094 - val_accuracy: 0.5834
Epoch 7/20
26270/26270 

Epoch 13/20
26270/26270 [==============================] - 11s 423us/step - loss: 1.9904 - accuracy: 0.4126 - val_loss: 2.9363 - val_accuracy: 0.1915
Epoch 14/20
26270/26270 [==============================] - 11s 425us/step - loss: 2.5002 - accuracy: 0.2923 - val_loss: 1.7304 - val_accuracy: 0.4792
Epoch 15/20
26270/26270 [==============================] - 11s 425us/step - loss: 2.0306 - accuracy: 0.4064 - val_loss: 2.1114 - val_accuracy: 0.3777
Epoch 16/20
26270/26270 [==============================] - 11s 432us/step - loss: 1.5027 - accuracy: 0.5442 - val_loss: 1.1852 - val_accuracy: 0.6237
Epoch 17/20
26270/26270 [==============================] - 11s 431us/step - loss: 1.2322 - accuracy: 0.6083 - val_loss: 1.1600 - val_accuracy: 0.6333
Epoch 18/20
26270/26270 [==============================] - 13s 499us/step - loss: 1.5515 - accuracy: 0.5274 - val_loss: 2.1130 - val_accuracy: 0.3697
Epoch 19/20
26270/26270 [==============================] - 12s 441us/step - loss: 1.8282 - accuracy:

The optimal learning rate seems to be around 0.125 (which is what similar to our original value that we were using to begin with). With that found, we will try to find the best matching momentum value and number of epochs. 

In [8]:
loss, accuracy = model.evaluate(x_tst, y_tst)
print('Test accuracy: %.4f' % (accuracy))

12939/12939 [==============================] - 2s 134us/step
Test accuracy: 0.0531


In [12]:
for m in momentums:
    print('momentum:', learning_rates[i])
    model = build_model(activation = 'sigmoid', batch_normalization = False)  # try 'relu' too
    opt = tf.keras.optimizers.SGD(learning_rate = 0.125, momentum = m, nesterov=False)
    model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
    model.fit(x_tr, y_tr, epochs = 20, validation_data= (x_tst, y_tst))
    print('========================================================================================================')

momentum: 0.5
Train on 26270 samples, validate on 12939 samples
Epoch 1/20
26270/26270 [==============================] - 11s 404us/step - loss: 2.8736 - accuracy: 0.2022 - val_loss: 1.9894 - val_accuracy: 0.3722
Epoch 2/20
26270/26270 [==============================] - 10s 398us/step - loss: 1.9140 - accuracy: 0.3903 - val_loss: 1.7960 - val_accuracy: 0.4192
Epoch 3/20
26270/26270 [==============================] - 12s 438us/step - loss: 1.6211 - accuracy: 0.4773 - val_loss: 1.4335 - val_accuracy: 0.5325
Epoch 4/20
26270/26270 [==============================] - 11s 422us/step - loss: 1.2838 - accuracy: 0.5953 - val_loss: 1.1001 - val_accuracy: 0.6472
Epoch 5/20
26270/26270 [==============================] - 11s 403us/step - loss: 0.9881 - accuracy: 0.6978 - val_loss: 0.8241 - val_accuracy: 0.7542
Epoch 6/20
26270/26270 [==============================] - 11s 402us/step - loss: 0.8455 - accuracy: 0.7402 - val_loss: 0.7181 - val_accuracy: 0.7850
Epoch 7/20
26270/26270 [==================

26270/26270 [==============================] - 11s 412us/step - loss: 0.4624 - accuracy: 0.8662 - val_loss: 0.5089 - val_accuracy: 0.8539
Epoch 14/20
26270/26270 [==============================] - 11s 414us/step - loss: 0.4664 - accuracy: 0.8651 - val_loss: 0.4147 - val_accuracy: 0.8805
Epoch 15/20
26270/26270 [==============================] - 11s 412us/step - loss: 0.3635 - accuracy: 0.8968 - val_loss: 0.3397 - val_accuracy: 0.9042
Epoch 16/20
26270/26270 [==============================] - 11s 411us/step - loss: 0.3372 - accuracy: 0.9046 - val_loss: 0.3590 - val_accuracy: 0.8949
Epoch 17/20
26270/26270 [==============================] - 11s 408us/step - loss: 0.2848 - accuracy: 0.9214 - val_loss: 0.2603 - val_accuracy: 0.9277
Epoch 18/20
26270/26270 [==============================] - 11s 413us/step - loss: 0.2688 - accuracy: 0.9254 - val_loss: 0.2604 - val_accuracy: 0.9287
Epoch 19/20
26270/26270 [==============================] - 11s 415us/step - loss: 0.2776 - accuracy: 0.9232 - va

KeyboardInterrupt: 

In [136]:
model = build_model(activation = 'sigmoid', batch_normalization = True)  # try 'relu' too
opt = tf.keras.optimizers.SGD(learning_rate = 0.1, momentum = 0, nesterov=True)
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
model.fit(x_tr, y_tr, epochs = 20, validation_data= (x_tst, y_tst))

Train on 31367 samples, validate on 7842 samples
Epoch 1/20
31367/31367 [==============================] - 19s 612us/step - loss: 2.0890 - accuracy: 0.4423 - val_loss: 1.3371 - val_accuracy: 0.5959
Epoch 2/20
31367/31367 [==============================] - 19s 612us/step - loss: 1.0283 - accuracy: 0.7164 - val_loss: 0.7710 - val_accuracy: 0.7828
Epoch 3/20
31367/31367 [==============================] - 19s 612us/step - loss: 0.7648 - accuracy: 0.7876 - val_loss: 0.7003 - val_accuracy: 0.8021
Epoch 4/20
31367/31367 [==============================] - 19s 620us/step - loss: 0.6282 - accuracy: 0.8189 - val_loss: 1.5658 - val_accuracy: 0.5598
Epoch 5/20
31367/31367 [==============================] - 19s 620us/step - loss: 0.5488 - accuracy: 0.8405 - val_loss: 0.4858 - val_accuracy: 0.8616
Epoch 6/20
31367/31367 [==============================] - 19s 619us/step - loss: 0.4619 - accuracy: 0.8675 - val_loss: 0.6529 - val_accuracy: 0.7989
Epoch 7/20
31367/31367 [==============================] -

In [137]:
yhat = model.predict(xtest0, verbose = False)

print(yhat)
print(len(yhat))


y_TST = [np.argmax(yhat[i]) for i in range(len(yhat))]

sub = [[0 for j in range(44)] for i in range(len(y_TST))]

for i in range(len(y_TST)):
    j = y_TST[i]+1
    sub[i][0] = i
    sub[i][j] = 1
# print(min(y_TST))
# print(max(y_TST))

print(y_TST[2])

[[7.3597480e-06 1.1880704e-06 4.0130180e-07 ... 9.2217283e-07
  4.8400634e-06 8.4510185e-07]
 [1.4789744e-04 9.7446352e-01 1.5995530e-03 ... 7.1707946e-06
  3.6577674e-08 4.9049589e-07]
 [1.3829646e-06 1.9243575e-07 3.6817747e-07 ... 7.9551297e-05
  1.6468884e-05 1.5074387e-06]
 ...
 [3.5664780e-04 1.0475375e-03 1.5668098e-03 ... 8.8161137e-04
  2.4159157e-04 4.5088156e-05]
 [1.3376473e-03 3.8526282e-03 5.0244057e-03 ... 5.3995554e-03
  1.0639826e-05 3.9452375e-04]
 [5.7629490e-04 9.8339608e-03 1.2270998e-03 ... 2.1267186e-03
  3.5112803e-03 2.1784692e-03]]
12630
38


In [149]:
model = build_model(activation = 'sigmoid', batch_normalization = True)  # try 'relu' too
opt = tf.keras.optimizers.SGD(learning_rate = 1e-5, momentum = 0.99, nesterov=True)
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
model.fit(x_tr, y_tr, epochs = 100, validation_data= (x_tst, y_tst))

Train on 31367 samples, validate on 7842 samples
Epoch 1/100
31367/31367 [==============================] - 20s 645us/step - loss: 3.6727 - accuracy: 0.0841 - val_loss: 3.4529 - val_accuracy: 0.1729
Epoch 2/100
31367/31367 [==============================] - 20s 638us/step - loss: 3.3240 - accuracy: 0.2169 - val_loss: 3.2240 - val_accuracy: 0.2437
Epoch 3/100
31367/31367 [==============================] - 20s 642us/step - loss: 3.1247 - accuracy: 0.2594 - val_loss: 3.0517 - val_accuracy: 0.2686
Epoch 4/100
31367/31367 [==============================] - 20s 643us/step - loss: 2.9623 - accuracy: 0.2944 - val_loss: 2.8987 - val_accuracy: 0.3044
Epoch 5/100
31367/31367 [==============================] - 20s 643us/step - loss: 2.8187 - accuracy: 0.3276 - val_loss: 2.7624 - val_accuracy: 0.3374
Epoch 6/100
31367/31367 [==============================] - 20s 645us/step - loss: 2.6932 - accuracy: 0.3510 - val_loss: 2.6442 - val_accuracy: 0.3590
Epoch 7/100
31367/31367 [==========================

Epoch 55/100
31367/31367 [==============================] - 20s 628us/step - loss: 0.9192 - accuracy: 0.8022 - val_loss: 0.9351 - val_accuracy: 0.7955
Epoch 56/100
31367/31367 [==============================] - 20s 632us/step - loss: 0.9034 - accuracy: 0.8073 - val_loss: 0.9420 - val_accuracy: 0.7812
Epoch 57/100
31367/31367 [==============================] - 20s 631us/step - loss: 0.8883 - accuracy: 0.8106 - val_loss: 0.9123 - val_accuracy: 0.8009
Epoch 58/100
31367/31367 [==============================] - 22s 687us/step - loss: 0.8701 - accuracy: 0.8157 - val_loss: 0.8870 - val_accuracy: 0.8064
Epoch 59/100
31367/31367 [==============================] - 20s 631us/step - loss: 0.8549 - accuracy: 0.8206 - val_loss: 0.8811 - val_accuracy: 0.8043
Epoch 60/100
31367/31367 [==============================] - 20s 631us/step - loss: 0.8404 - accuracy: 0.8250 - val_loss: 0.8548 - val_accuracy: 0.8203
Epoch 61/100
31367/31367 [==============================] - 20s 629us/step - loss: 0.8247 - ac

KeyboardInterrupt: 

In [138]:
import pandas as pd

cols = []
for i in range(43):
    cols.append('oh_'+str(i))
cols = ['id'] + cols

df = pd.DataFrame(data=sub, columns=cols)
df = df.set_index('id')

df.to_csv('results.csv', sep=',', encoding='utf-8')
df

,oh_0,oh_1,oh_2,oh_3,oh_4,oh_5,oh_6,oh_7,oh_8,oh_9,...,oh_33,oh_34,oh_35,oh_36,oh_37,oh_38,oh_39,oh_40,oh_41,oh_42
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12625,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12626,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
12627,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Results


learning rate: 0.0001

momentum: 0.99

number epochs: 200

Although the number of epochs is a bit high, having such a large epoch range allows our model to slowly increase accuracy as the epoch step increases. If we limit our number of epochs to 20, our model settles around 87% accuracy. If we increase epochs to 50, our model accuracy incrases to 90% or so. Increasing it further to 100 brings us close to 95% accuracy. Finally, 200 epochs brings us close to 97% accuracy.

Overall, these results are similar to previous hyperparamter tuning we've performed in the past. By setting a low learning rate (with decent momentum to keep our model "learning" through each epoch step) and allowing our model to run through a high number of epochs, our model can steadily reduce its error until its performance plateaus.